In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { BufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";

const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});

const memory = new BufferMemory();
const chain = new ConversationChain({ llm: chatModel, memory,verbose:true });
const res1 = await chain.invoke({ input: "我是小明" });


In [ ]:
res1

In [ ]:
const res2 = await chain.invoke({ input: "我叫什么？" });


In [ ]:
res2

In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { BufferWindowMemory } from "langchain/memory";

const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});

const memory = new BufferWindowMemory({ k: 1 });
const chain = new ConversationChain({ llm: chatModel, memory, verbose: true });


In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { ConversationSummaryMemory } from "langchain/memory";
import { PromptTemplate } from "@langchain/core/prompts";
import { ConversationChain } from "langchain/chains";



const model = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});

const memory = new ConversationSummaryMemory({
  memoryKey: "summary",
  llm: model,
});

const prompt = PromptTemplate.fromTemplate(`
    你是一个乐于助人的助手。尽你所能回答所有问题。
    
    这是聊天记录的摘要:
    {summary}
    Human: {input}
    AI:`);

const chain = new ConversationChain({
  llm: model,
  prompt,
  memory,
  verbose: true,
});

const res1 = await chain.invoke({ input: "我是小明" });
const res2 = await chain.invoke({ input: "我叫什么？" });


In [ ]:
res1
res2

In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { ConversationChain } from "langchain/chains";

const model = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});

const memory = new ConversationSummaryBufferMemory({
  llm: model,
  maxTokenLimit: 200,
});

const chain = new ConversationChain({ llm: model, memory, verbose: true });


In [ ]:
// main.ts
import { ConversationChain } from "langchain/chains";
import { ChatOpenAI } from "@langchain/openai";
import { ConversationSummaryMemory } from "langchain/memory";
import { BufferWindowMemory } from "langchain/memory";
import { PromptTemplate } from "@langchain/core/prompts";

/* ---------- LLM ---------- */
const llm = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});

/* ---------- 记忆 ---------- */
// 1) 最近 3 轮原始对话
const recentMemory = new BufferWindowMemory({
  k: 3, // 保留最近 3 组（人+AI）
  memoryKey: "recent_history",
  inputKey: "input",
});

// 2) 持续更新的整个对话摘要
const summaryMemory = new ConversationSummaryMemory({
  llm, // 用于生成摘要
  memoryKey: "summary",
  inputKey: "input",
});

/* ---------- Prompt ---------- */
const prompt = PromptTemplate.fromTemplate(
  `Summary of entire conversation:
{summary}

Recent messages (last 3 turns):
{recent_history}

Human: {input}
AI:`
);

/* ---------- 链 ---------- */
const chain = new ConversationChain({
  llm,
  prompt,
  memory: {
    // 多记忆合并
    loadMemoryVariables: async (inputs) => {
      const [summary, recent] = await Promise.all([
        summaryMemory.loadMemoryVariables(inputs),
        recentMemory.loadMemoryVariables(inputs),
      ]);
      return { ...summary, ...recent };
    },
    saveContext: async (inputs, outputs) => {
      // 同时写两份记忆
      await Promise.all([
        summaryMemory.saveContext(inputs, outputs),
        recentMemory.saveContext(inputs, outputs),
      ]);
    },
  },
});

/* ---------- 测试 ---------- */
await chain.call({ input: "你好，我叫 Kai" });
await chain.call({ input: "我喜欢粉色玫瑰" });
await chain.call({ input: "记住，下周我姐姐生日" });
await chain.call({ input: "我昨天说了什么？" }); // 模型能结合摘要和最近三轮回答


In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import {
  EntityMemory,
  ENTITY_MEMORY_CONVERSATION_TEMPLATE,
} from "langchain/memory";
import { ConversationChain } from "langchain/chains";

const model = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
  verbose: true,
});

const memory = new EntityMemory({
  llm: new ChatOpenAI({
    configuration: {
      baseURL: "https://api.moonshot.cn/v1",
      apiKey: process.env.OPENAI_API_KEY,
    },
    modelName: "kimi-k2-0711-preview",
    verbose: true,
    verbose: true,
  }),
  chatHistoryKey: "history",
  entitiesKey: "entities",
});
const chain = new ConversationChain({
  llm: model,
  prompt: ENTITY_MEMORY_CONVERSATION_TEMPLATE,
  memory: memory,
  verbose: true,
});


In [ ]:
const res1 = await chain.call({ input: "我叫小明，今年 18 岁" });


In [ ]:
const res2 = await chain.call({
  input: "ABC 是一家互联网公司，主要是售卖方便面的公司",
});


In [ ]:
const res3 = await chain.call({ input: "介绍小明" });


In [ ]:
const res3 = await chain.call({ input: "介绍小明和 ABC" });


In [ ]:
const test1 = await memory.loadMemoryVariables({
  input: "介绍小明和 ABC",
});